# Initialization

Test notebook for the DataHandlers. Test the CMAPSS DataHandler

First we import the necessary packages and create the global variables.

In [1]:
import numpy as np
import math
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D, LSTM
from keras.optimizers import Adam, SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

import CMAPSAuxFunctions

from numpy.random import seed
from tensorflow import set_random_seed


Using TensorFlow backend.


# Define different types of Keras models

In [2]:
l2_lambda_regularization = 0
#l1_lambda_regularization = 0.2

def RULmodel_LSTM(input_shape):
    """Define the RNN model"""
    
    #Create a sequential model
    model = Sequential()
    #model.add(Masking(mask_value=0, imput))
    #model.add(LSTM(input_shape=input_shape, units=100, return_sequences=True, name='lstm1')))
    model.add(LSTM(input_shape=input_shape, units=20, return_sequences=False, name='lstm2'))
    model.add(Dense(10, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model
    

def RULmodel_SN_5(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc1'))
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), 
                    name='fc2'))
    model.add(Dense(1, activation='linear', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

def test_model(input_shape):
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', 
                    kernel_initializer=keras.initializers.glorot_normal(seed=0), 
                    kernel_regularizer=regularizers.l2(l2_lambda_regularization), name='out'))
    
    return model

In [3]:
def print_model_weights(model):

    for layer in model.layers:
        weights = layer.get_weights() # list of numpy arrays
        
        for weight in weights:
        
            print(weight.shape)
            print(weight)

# Create the Data Handler

In [4]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, window_stride)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create Tunable Model

In [5]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [6]:
def get_compiled_model(shape, model_type='ann'):
    
    K.clear_session()  #Clear the previous tensorflow graph
    
    #To test the model without randomness
    seed(0)
    set_random_seed(0)
    
    #Shared parameters for the models
    optimizer = Adam(lr=0.001)
    
    #optimizer = SGD(lr=0.001, momentum=0.0, decay=0.0, nesterov=False)
    
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = test_model(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        model = RULmodel_LSTM(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    else:
        pass

    return model

In [14]:
num_features = len(selected_features)

shape = len(selected_features)*window_size
model = get_compiled_model(shape, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', model, lib_type='keras', data_handler=dHandler_cmaps)

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', model, lib_type='keras', data_handler=dHandler_cmaps)
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

# Load Data

In [8]:
tModel.data_handler.sequence_length = 30
#tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
tModel.data_handler.sequence_stride = 1
tModel.data_handler.max_rul = 128

In [9]:
#Load Non sequenced data (unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = None
tModel.data_scaler = min_max_scaler

tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

Loading data for dataset 1 with window_size of 30, stride of 1 and maxRUL of 128. Cros-Validation ratio 0
Loading data from file and computing dataframes
Printing shapes

Training data (X, y)
(17731, 420)
(17731,)
Testing data (X, y)
(100, 420)
(100,)
Printing first 5 elements

Training data (X, y)
[[-0.58075601 -0.0455243  -0.27982732 ..., -0.81818182  0.43307087
   0.4679733 ]
 [-0.35395189  0.0629156  -0.18014129 ..., -0.45454545  0.25984252
   0.25294702]
 [-0.21649485 -0.13299233 -0.13854003 ..., -0.45454545  0.38582677
   0.72049425]
 [-0.21649485 -0.39897698 -0.2299843  ..., -0.45454545  0.08661417
   0.29640676]
 [-0.20274914 -0.39590793 -0.05926217 ..., -0.45454545  0.05511811
   0.17880983]]
[ 128.  128.  128.  128.  128.]
Testing data (X, y)
[[-0.65635739 -0.10946292 -0.48312402 ..., -0.27272727  0.05511811
   0.30947309]
 [ 0.03780069 -0.07365729 -0.27629513 ..., -0.63636364  0.05511811
   0.04416986]
 [ 0.13402062 -0.08644501  0.038854   ...,  0.09090909  0.24409449
   0.0

In [ ]:
#Load sequenced data (do not unroll sequence into a single feature vector)

tModel.data_handler.data_scaler = min_max_scaler
tModel.data_scaler = None

tModel.load_data(unroll=False, verbose=1, cross_validation_ratio=0)
#tModel.data_handler.print_data()
tModel.print_data()

In [ ]:
#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = None

# Train the model and test some Tunable Model functionalities

In [10]:
nFeatures = len(selected_features)

#lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)

shape = len(selected_features*window_size)

#modelRULSN = RULmodel_SN_5(shape)
#modelRULSN.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)

model = get_compiled_model(shape, model_type='ann')
tModel.change_model('ModelRUL_SN_1', model, 'keras')
#tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')

#tModel.print_data()

#shape = (window_size, num_features)
#model = get_compiled_model(shape, model_type='lstm')
#tModel.change_model('ModelRUL_RNN_1', model, 'keras')

#tModel.data_handler.data_scaler = min_max_scaler
#tModel.data_scaler = min_max_scaler

print("Printing model weights")
print_model_weights(tModel.model)

tModel.epochs = 20
tModel.train_model(verbose=1)

Printing model weights
(420, 20)
[[ 0.00835461 -0.12109794  0.02095141 ...,  0.03727926  0.10370639
   0.00207878]
 [-0.06049565  0.09017985 -0.01698299 ...,  0.04146439  0.03719759
  -0.09819514]
 [ 0.04680334  0.02662419  0.02083843 ...,  0.04953294  0.06038643
   0.09780484]
 ..., 
 [-0.06602989  0.01178172  0.01025762 ...,  0.06893838 -0.0499379
   0.01730988]
 [ 0.00335656 -0.00574053  0.04053649 ..., -0.07136186 -0.02316877
   0.08826228]
 [-0.00064016  0.10836472 -0.04717498 ...,  0.0050592  -0.02888336
   0.03074028]]
(20,)
[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.]
(20, 1)
[[ 0.03824222]
 [-0.55431098]
 [ 0.09590249]
 [-0.40720478]
 [ 0.17107496]
 [-0.29680154]
 [-0.1824448 ]
 [-0.41047722]
 [-0.31966969]
 [ 0.15358827]
 [-0.2766681 ]
 [-0.36381829]
 [-0.31770143]
 [ 0.20868248]
 [ 0.30954045]
 [-0.37831017]
 [ 0.57799178]
 [ 0.17064126]
 [ 0.47470331]
 [ 0.00951534]]
(1,)
[ 0.]
Epoch 1/20
17731/17731 [==============================] - 0

In [21]:
tModel.evaluate_model(['rhs', 'rmse'], round=2)
print("scores")

cScores = tModel.scores
rmse = math.sqrt(cScores['score_1'])
rmse2 = cScores['rmse']
rhs = cScores['rhs']
time = tModel.train_time

i = range(1,len(tModel.y_test)+1)


for i, rul_prediction, rul_prediction_rounded, true_rul in zip(i, tModel.y_predicted, tModel.y_predicted_rounded, tModel.y_test):
    print('Engine {}, Predicted RUL {}, Rounded RUL {}, Real RUL {}'.format(i, rul_prediction, 
                                                                    rul_prediction_rounded, 
                                                                    true_rul))

print(cScores)
print("RMSE: {}".format(rmse))
print("RMSE2: {}".format(rmse2))
print("RHS: {}".format(rhs))
print("Time : {} seconds".format(time))

100/100 [==============================] - 0s 701us/step
scores
Engine 1, Predicted RUL [ 110.47058868], Rounded RUL 110.0, Real RUL 112.0
Engine 2, Predicted RUL [ 129.27981567], Rounded RUL 129.0, Real RUL 98.0
Engine 3, Predicted RUL [ 48.54970551], Rounded RUL 48.0, Real RUL 69.0
Engine 4, Predicted RUL [ 82.32772064], Rounded RUL 82.0, Real RUL 82.0
Engine 5, Predicted RUL [ 105.29595184], Rounded RUL 105.0, Real RUL 91.0
Engine 6, Predicted RUL [ 111.28095245], Rounded RUL 111.0, Real RUL 93.0
Engine 7, Predicted RUL [ 101.98551178], Rounded RUL 101.0, Real RUL 91.0
Engine 8, Predicted RUL [ 95.60164642], Rounded RUL 95.0, Real RUL 95.0
Engine 9, Predicted RUL [ 121.45348358], Rounded RUL 121.0, Real RUL 111.0
Engine 10, Predicted RUL [ 91.95681763], Rounded RUL 91.0, Real RUL 96.0
Engine 11, Predicted RUL [ 78.26259613], Rounded RUL 78.0, Real RUL 97.0
Engine 12, Predicted RUL [ 98.55871582], Rounded RUL 98.0, Real RUL 124.0
Engine 13, Predicted RUL [ 74.36135101], Rounded RUL 7